In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install accelerate>=0.12.0
!pip install datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/en2sparql

Mounted at /content/drive
/content/drive/MyDrive/en2sparql


In [3]:
import torch
import json
from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-3b", padding_side="left")

dolly = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")
raw_datasets = load_dataset("orkg/SciQA")
print(raw_datasets)


def divide_chunks(l_, n_):
    for i_ in range(0, len(l_), n_):
        yield l_[i_:i_ + n_]


def clean(st):
    st = st.replace("\n", " ")
    st = st.replace("?", " ?")
    st = st.replace("{", " { ")
    st = st.replace("}", " } ")
    st = st.replace("\\'", "'")

    while "  " in st:
        st = st.replace("  ", " ")
    return st


def get_key(q):
    t0 = q.get('template_id')
    if t0 is None:
        t0 = "None"
    t = str(q.get("number_of_patterns")) + "-" + t0
    return t


def get_keys(n_):
    train = raw_datasets.get("train")
    patterns = {}
    for q in train:
        t = get_key(q)
        query = clean(q["query"]["sparql"])
        question = q["question"]["string"]
        if t not in patterns:
            patterns[t] = [[query, question, len(query)]]
        else:
            patterns[t].append([query, question, len(query)])

    for t in patterns:
        code = patterns.get(t)
        code = sorted(code, key=lambda x: x[2], reverse=True)
        patterns[t] = code[:n_]
    return patterns


def prepare_queries(n_):
    keys = get_keys(n_)
    data = raw_datasets.get("test")
    queries = []
    for q in data:
        t = get_key(q)
        question = q["question"]["string"]
        suggestion = keys.get(t)
        if suggestion is None:
            print("Error with key", t)
            queries.append("translate the following English text '" + question + "' to a sparql query")
        else:
            final_q = ""
            for i_, k in enumerate(suggestion):
                final_q += "\n input (English text): " + k[1]
                final_q += "\n output (Sparql query): " + k[0]
            final_q += "\n input (English text): " + question
            final_q += "\n output (Sparql query): "
            queries.append(final_q)
    return queries

shots = 2
query_list = prepare_queries(shots)

print(len(query_list))


n = 5
q_list = list(divide_chunks(query_list, n))
sparql = [clean(x["query"]["sparql"]) for x in raw_datasets.get("test")]

gs = []
lens =[]

i = 0

for group in q_list:
    print(str(i) + "%", end="  ")
    i += 1/len(q_list)*100

    res_ = [tokenizer.encode(question) for question in group]
    len_ = [len(x) for x in res_]
    lens += len_

    res = dolly(group)
    gst = [x[0]["generated_text"] for x in res]
    print(gst)
    gs += gst

print([x for x in lens if x>2048])

result = {"questions": query_list, "sparql": sparql, "generated_sparql": gs, "prompt_len": lens}

with open("dolly_"+str(shots)+"_shot_results_tok.json", "w", encoding="utf-8") as text_file:
    print(json.dumps(result), file=text_file)


A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 1795
    })
    validation: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 257
    })
    test: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 513
    })
})
Error with key 10-None
513
0%  ['The best performing model is the Transformer model from the Neural Language Model Research Group at the University of Massachusetts, Amherst. The Transformer model achieved an accuracy score of 99.0%.', 'SELECT?paper?paper_lbl WHERE { \n ?dataset a orkgc:Dataset; \n  rdfs:label?datase

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


['SELECT DISTINCT?metric?metric_lbl (MAX(?value) AS?score) WHERE { { SELECT?metric?metric_lbl?value WHERE {?dataset a orkgc:Dataset; rdfs:label?dataset_lbl. FILTER (str(?dataset_lbl) = "AESLC")?benchmark orkgp:HAS_DATASET?dataset; orkgp:HAS_EVALUATION?eval.?eval orkgp:HAS_VALUE?value. OPTIONAL {?eval orkgp:HAS_METRIC?metric.?metric rdfs:label?metric_lbl. }?cont orkgp:HAS_BENCHMARK?benchmark. OPTIONAL {?cont orkgp:HAS_MODEL?model.?model rdfs:label?model_lbl. } } ORDER BY DESC(?value) } } GROUP BY?metric?metric_lbl', 'SELECT DISTINCT?paper?paper_lbl WHERE {?dataset a orkgc:Dataset; rdfs:label?dataset_lbl. FILTER (str(?dataset_lbl) = "Automatically labeled Medline abstracts corpus")?benchmark orkgp:HAS_DATASET?dataset.?cont orkgp:HAS_BENCHMARK?benchmark.?paper orkgp:P31?cont; rdfs:label?paper_lbl. }', 'SELECT DISTINCT?metric?metric_lbl (MAX(?value) AS?score) WHERE { { SELECT?metric?metric_lbl?value WHERE {?dataset a orkgc:Dataset; wos:Synset?dataset_synset. FILTER (str(?dataset_synset) = 

KeyboardInterrupt: ignored